In [ ]:
import pandas as pd
import numpy as np
import joblib
from google.cloud import storage

BUCKET_NAME = "bucket_name"
DATA_PATH = f"gs://{BUCKET_NAME}/Normalized_Dataset.csv"
MODEL_PATH = f"gs://{BUCKET_NAME}/LinUCB_Model_Final.pkl"
TRAINED_MODEL_PATH = f"gs://{BUCKET_NAME}/Trained_LinUCB_Model_Final.pkl"

df = pd.read_csv(DATA_PATH)

# FEATURE ENGINEERING & PREPROCESSING
features = df.drop(columns=["Feedback", "Food_ID", "Food_Name", "Restaurant_ID", "Restaurant_Name", "User_ID"]).values
target = df["Feedback"].values

class LinUCB:
    def __init__(self, alpha, num_arms, num_features):
        self.alpha = alpha
        self.num_arms = num_arms
        self.A = [np.identity(num_features) for _ in range(num_arms)]
        self.b = [np.zeros((num_features, 1)) for _ in range(num_arms)]

    def select_arm(self, x):
        """ Selects the arm using Upper Confidence Bound (UCB) """
        p = []
        for arm in range(self.num_arms):
            A_inv = np.linalg.inv(self.A[arm] + 1e-6 * np.identity(len(x))) # Regularization
            theta = np.dot(A_inv, self.b[arm])
            ucb = np.dot(theta.T, x) + self.alpha * np.sqrt(np.dot(x.T, np.dot(A_inv, x)) + 1e-6)  # Avoid zero error
            p.append(ucb)
        return np.argmax(p)  # Arm with highest confidence bound

    def train(self, X, y):
        for i in range(len(X)):
            x = X[i].reshape(-1, 1)
            arm = self.select_arm(x)
            self.A[arm] += np.outer(x, x)  # Update matrix
            self.b[arm] += y[i].reshape(-1, 1) * x  # Ensure correct shape

    def save_model(self, gcs_path):
        """ Save model to Cloud Storage """
        storage_client = storage.Client()
        bucket = storage_client.bucket(BUCKET_NAME)
        blob = bucket.blob(gcs_path.split("/")[-1])

        local_model_path = "/tmp/" + gcs_path.split("/")[-1]
        joblib.dump(self, local_model_path)
        blob.upload_from_filename(local_model_path)

        print(f"Model saved to: {gcs_path}")

    @staticmethod
    def load_model(gcs_path):
        """ Load model from Cloud Storage """
        storage_client = storage.Client()
        bucket = storage_client.bucket(BUCKET_NAME)
        blob = bucket.blob(gcs_path.split("/")[-1])

        local_model_path = "/tmp/" + gcs_path.split("/")[-1]
        blob.download_to_filename(local_model_path)
        model = joblib.load(local_model_path)

        print("Model successfully loaded.")
        return model

# LOAD OR INITIALIZE MODEL
try:
    model = LinUCB.load_model(MODEL_PATH)
    print("Existing model loaded from GCP.")
except Exception as e:
    print("No existing model found. Initializing a new model.")
    model = LinUCB(alpha=0.1, num_arms=features.shape[1], num_features=features.shape[1])

# ==== TRAIN MODEL ====
print("Training model...")
model.train(features, target)

# ==== SAVE TRAINED MODEL ====
model.save_model(TRAINED_MODEL_PATH)